In [ ]:

import gym
import torch
from TenGene1 import register_env

from enum import IntEnum
from typing import Tuple, Optional, List
import numpy as np
import numpy
from itertools import product
import pickle

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

register_env()
env = gym.make('TenGene1-v0')

number_of_states = 10
noise_num = 0.001
dna_dsb = np.array(np.mat('0 0 0 0 0 0 0 0 0 0'))

nums_missing = 5

stateNum = 2**number_of_states  # 0-1023, 1024 in total
modelNum = 3**nums_missing  # 243 models, with 5 unknown
qNum = 10  # 10 qs needed

with open('GRNBOVals10Gene5MissingAllMissingOnes.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    ConnectivitySpace = pickle.load(f)

combs = product(range(2), repeat=number_of_states)
combs = list(combs)
A = numpy.array(combs)

class Action(IntEnum):

    GENE1 = 0
    GENE2 = 1
    GENE3 = 2
    GENE4 = 3
    GENE5 = 4
    GENE6 = 5
    GENE7 = 6
    GENE8 = 7
    GENE9 = 8
    GENE10 = 9

our_action_size = 10

score = 0
random_score = 0
Noperturb_score = 0

our_maxcount = 0
random_maxcount = 0
Noperturb_maxcount = 0

horizon = 50

notmax_pis_random = np.zeros((1000,horizon))

our_posteriors = np.zeros((1000, horizon, modelNum))
max_pis_random_entropy = np.zeros((1000,horizon))

max_pis_ours = np.zeros((1000,horizon))
max_pis_random = np.zeros((1000,horizon))
max_pis_NoPerturb = np.zeros((1000,horizon))
##################################################3
q1_1_ours = np.zeros((1000,horizon))
q1_2_ours = np.zeros((1000,horizon))
q1_3_ours = np.zeros((1000,horizon))

q2_1_ours = np.zeros((1000,horizon))
q2_2_ours = np.zeros((1000,horizon))
q2_3_ours = np.zeros((1000,horizon))

q3_1_ours = np.zeros((1000,horizon))
q3_2_ours = np.zeros((1000,horizon))
q3_3_ours = np.zeros((1000,horizon))

q4_1_ours = np.zeros((1000,horizon))
q4_2_ours = np.zeros((1000,horizon))
q4_3_ours = np.zeros((1000,horizon))

q5_1_ours = np.zeros((1000,horizon))
q5_2_ours = np.zeros((1000,horizon))
q5_3_ours = np.zeros((1000,horizon))
########################
q1_1_random  = np.zeros((1000,horizon))
q1_2_random = np.zeros((1000,horizon))
q1_3_random = np.zeros((1000,horizon))

q2_1_random = np.zeros((1000,horizon))
q2_2_random = np.zeros((1000,horizon))
q2_3_random = np.zeros((1000,horizon))

q3_1_random = np.zeros((1000,horizon))
q3_2_random = np.zeros((1000,horizon))
q3_3_random = np.zeros((1000,horizon))

q4_1_random = np.zeros((1000,horizon))
q4_2_random = np.zeros((1000,horizon))
q4_3_random = np.zeros((1000,horizon))

q5_1_random = np.zeros((1000,horizon))
q5_2_random = np.zeros((1000,horizon))
q5_3_random = np.zeros((1000,horizon))
###################
q1_1_NoPerturb = np.zeros((1000,horizon))
q1_2_NoPerturb = np.zeros((1000,horizon))
q1_3_NoPerturb = np.zeros((1000,horizon))

q2_1_NoPerturb = np.zeros((1000,horizon))
q2_2_NoPerturb = np.zeros((1000,horizon))
q2_3_NoPerturb = np.zeros((1000,horizon))

q3_1_NoPerturb = np.zeros((1000,horizon))
q3_2_NoPerturb = np.zeros((1000,horizon))
q3_3_NoPerturb = np.zeros((1000,horizon))

q4_1_NoPerturb = np.zeros((1000,horizon))
q4_2_NoPerturb = np.zeros((1000,horizon))
q4_3_NoPerturb = np.zeros((1000,horizon))

q5_1_NoPerturb = np.zeros((1000,horizon))
q5_2_NoPerturb = np.zeros((1000,horizon))
q5_3_NoPerturb = np.zeros((1000,horizon))


def step_forAvg(agent_pos, action: Action) -> Tuple[Tuple, float, bool, dict]:

    mycombs = product(range(3), repeat=nums_missing)
    Combinations = np.float64(numpy.array(list(mycombs)) - 1)

    for u in range(nums_missing):
        numpy.place(Combinations[:, u], Combinations[:, u] == -1, (agent_pos)[u*2+0+number_of_states])
        numpy.place(Combinations[:, u], Combinations[:, u] == 0, (agent_pos)[u*2+1+number_of_states])
        numpy.place(Combinations[:, u], Combinations[:, u] == 1, 1-(agent_pos)[u*2+0+number_of_states]-(agent_pos)[u*2+1+number_of_states])

    model_probs = np.ones(modelNum)

    for v in range(nums_missing):
        model_probs = np.multiply(Combinations[:, v], model_probs)

    numpy.place(model_probs, model_probs<0, 0)
    model_probs = model_probs/np.sum(model_probs)
    model_probs = numpy.nan_to_num(model_probs, copy=True, nan=0.0, posinf=None, neginf=None)
    if np.sum(model_probs) == 0:
        model_probs = (1/modelNum)*np.ones(modelNum)

    argmaxes = np.argwhere(model_probs == np.amax(model_probs))
    argmaxes_one_d = argmaxes[:, 0]
    model_choice = np.random.choice(argmaxes_one_d)

    Cnext = np.atleast_2d(ConnectivitySpace[model_choice, :])
    CnextReshape = np.reshape(Cnext, (number_of_states, number_of_states))

    x_old = np.zeros(number_of_states)
    for t in range(number_of_states):
        x_old[t] = int((agent_pos)[t])
    x_old = np.atleast_2d(x_old)
    x_old.reshape((1,number_of_states))
    x_new = 1 * (np.matmul(x_old, (CnextReshape.T) + dna_dsb) > 0)

    perturb = np.zeros(number_of_states)
    if action != number_of_states:
        perturb[action] = 1
    perturb.reshape((1,number_of_states))
    noise = np.random.binomial(1, noise_num, size=(1, number_of_states))

    x_new = 1 * np.logical_xor(x_new, perturb)
    x_new = 1 * np.logical_xor(x_new, noise)

    x_new_index = (A == x_new).all(axis=1).nonzero()
    x_new_index = np.array(x_new_index[0])

    M_indices = np.ones(modelNum)
    for j in range(modelNum):

        Cnext = np.atleast_2d(ConnectivitySpace[j, :])
        CnextReshape = np.reshape(Cnext, (number_of_states, number_of_states))

        x_expected = 1 * (np.matmul(x_old, (CnextReshape.T) + dna_dsb) > 0)
        x_expected = 1 * np.logical_xor(x_expected, perturb)

        for k in range(number_of_states):

            if x_expected[0, k] == x_new[0, k]:
                temp1 = 1 - noise_num
            else:
                temp1 = noise_num

            M_indices[j] = M_indices[j] * temp1

    model_probs_new = np.multiply(model_probs, M_indices)
    model_probs_new = model_probs_new/np.sum(model_probs_new) #normalized

    new_qvals = np.zeros(qNum)
    mycombs2 = product(range(3), repeat=nums_missing)
    Combinations2 = np.float64(numpy.array(list(mycombs2)) - 1)

    for w in range(nums_missing):
        indices = np.array(np.atleast_2d(Combinations2[:, w] == -1).all(axis=0).nonzero())
        new_qvals[2*w+0] = np.sum(model_probs_new[indices])

        indices = np.array(np.atleast_2d(Combinations2[:, w] == 0).all(axis=0).nonzero())
        new_qvals[2*w+1] = np.sum(model_probs_new[indices])

    done = False

    reward = np.amax(model_probs_new) - np.amax(model_probs)

    return reward

stat_of_actions20 = np.zeros(10)
stat_of_actions50 = np.zeros(10)

for i in range(1000):
    print(i)

    state = env.reset()

    state = np.array(state)
    count = 0

    while(1):

        Avg_res = np.zeros(our_action_size)

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE1)
        Avg_res[Action.GENE1] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE2)
        Avg_res[Action.GENE2] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE3)
        Avg_res[Action.GENE3] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE4)
        Avg_res[Action.GENE4] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE5)
        Avg_res[Action.GENE5] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE6)
        Avg_res[Action.GENE6] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE7)
        Avg_res[Action.GENE7] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE8)
        Avg_res[Action.GENE8] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE9)
        Avg_res[Action.GENE9] = sum_ent/1

        sum_ent = 0
        for avg_mont in range(1):
            sum_ent = sum_ent + step_forAvg(state, Action.GENE10)
        Avg_res[Action.GENE10] = sum_ent/1

        argmaxes = np.argwhere(Avg_res == np.amax(Avg_res))
        argmaxes_one_d = argmaxes[:, 0]
        random_argmaxes = np.random.choice(argmaxes_one_d)
        action = random_argmaxes
        
        if count < 20:
            stat_of_actions20[action] = stat_of_actions20[action] + 1
        
        stat_of_actions50[action] = stat_of_actions50[action] + 1

        next_state, reward, done, _ = env.step(action)

        state = next_state

        ###
        np_state = np.array(state)
        ###

        if done==1:
            random_maxcount = random_maxcount + 1


        random_score = random_score + reward

        model_probs = np.ones(modelNum)

        ############################################ here

        mycombs = product(range(3), repeat=nums_missing)
        Combinations = np.float64(numpy.array(list(mycombs)) - 1)

        q1_1_random[i,count] = next_state[0+0+number_of_states]
        q1_2_random[i,count] = next_state[0+1+number_of_states]
        q1_3_random[i,count] = 1-next_state[0+0+number_of_states]-next_state[0+1+number_of_states]

        q2_1_random[i,count] = next_state[2+0+number_of_states]
        q2_2_random[i,count] = next_state[2+1+number_of_states]
        q2_3_random[i,count] = 1-next_state[2+0+number_of_states]-next_state[2+1+number_of_states]

        q3_1_random[i,count] = next_state[4+0+number_of_states]
        q3_2_random[i,count] = next_state[4+1+number_of_states]
        q3_3_random[i,count] = 1-next_state[4+0+number_of_states]-next_state[4+1+number_of_states]

        q4_1_random[i,count] = next_state[6+0+number_of_states]
        q4_2_random[i,count] = next_state[6+1+number_of_states]
        q4_3_random[i,count] = 1-next_state[6+0+number_of_states]-next_state[6+1+number_of_states]

        q5_1_random[i,count] = next_state[8+0+number_of_states]
        q5_2_random[i,count] = next_state[8+1+number_of_states]
        q5_3_random[i,count]= 1-next_state[8+0+number_of_states]-next_state[8+1+number_of_states]

        for u in range(nums_missing):
            numpy.place(Combinations[:, u], Combinations[:, u] == -1, (next_state)[u*2+0+number_of_states])
            numpy.place(Combinations[:, u], Combinations[:, u] == 0, (next_state)[u*2+1+number_of_states])
            numpy.place(Combinations[:, u], Combinations[:, u] == 1, 1-(next_state)[u*2+0+number_of_states]-(next_state)[u*2+1+number_of_states])

        model_probs = np.ones(3**nums_missing)

        for v in range(nums_missing):
            model_probs = np.multiply(Combinations[:, v], model_probs)
        ############################################

        max_pis_random[i,count] = np.max(model_probs)
        
        if np.argmax(model_probs) == 222:
            notmax_pis_random[i,count] = 1
        else:
            notmax_pis_random[i,count] = 0  

        our_posteriors[i, count, :] = model_probs

        temp_entropy = 0
        for j in range(3**nums_missing):
            if model_probs[j] == 0:
                temp_entropy = temp_entropy
            else:
                temp_entropy = temp_entropy + model_probs[j] * np.log(model_probs[j])
        max_pis_random_entropy[i,count] = temp_entropy

        if done or count==(50-1):
            break

        count = count + 1

with open('10Gene5UnknownTestOnRealEnvAddedQvals_Entropy_MAP_Final_1.pkl','wb') as f:  # Python 3: open(..., 'wb')
     pickle.dump(max_pis_random_entropy, f)

with open('10Gene5UnknownTestOnRealEnvAddedQvals_MAP_Final_1.pkl','wb') as f:  # Python 3: open(..., 'wb')
     pickle.dump(max_pis_random, f)

with open('10Gene5UnknownTestOnRealEnvAddedQvals_Qvalues_MAP_Final_1.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([q1_1_random, q1_2_random, q1_3_random, q2_1_random, q2_2_random, q2_3_random, q3_1_random, q3_2_random, q3_3_random, q4_1_random, q4_2_random, q4_3_random, q5_1_random, q5_2_random, q5_3_random], f)

with open('10Gene5UnknownTestOnRealEnvActionAddedQvals_OurPosteriors_MAP_Final_1.pkl','wb') as f:  # Python 3: open(..., 'wb')
     pickle.dump(our_posteriors, f)

with open('MAPForTable_1.pkl','wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(notmax_pis_random, f)
    
with open('stat_of_actions_Until50_ver2_MAP.pkl','wb') as f:  # Python 3: open(..., 'wb')
     pickle.dump(stat_of_actions50, f)   
        
with open('stat_of_actions_Until20_ver2_MAP.pkl','wb') as f:  # Python 3: open(..., 'wb')
     pickle.dump(stat_of_actions20, f)   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
